The goal of this project is to predict house sale prices based on property features and understand which factors most strongly influence pricing decisions using machine learning models.

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor



data = pd.read_csv("G:\house-price-ml\Data/train.csv")
data.head()
data.shape()
data.info()


In [ ]:
y = data["SalePrice"]
X = data.drop("SalePrice", axis=1)


In [ ]:
numerical_cols = X.select_dtypes(include=["int64", "float64"]).columns
categorical_cols = X.select_dtypes(include=["object"]).columns

len(numerical_cols), len(categorical_cols)


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=42
)


The dataset is split into training and validation sets to evaluate how well the model generalizes to unseen data.